In [1]:
library(DBI)
library(dplyr)
#library(Hmisc)
library(ggplot2)
#library(foreign)
library(VIM)
library(ggExtra)
library(naniar)
library(mvoutlier)
library(MissMech);


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: colorspace

Loading required package: grid

VIM is ready to use.


Suggestions and bug-reports can be submitted at: https://github.com/statistikat/VIM/issues


Attaching package: 'VIM'


The following object is masked from 'package:datasets':

    sleep


Warning message:
"package 'ggExtra' was built under R version 4.0.2"
Warning message:
"package 'naniar' was built under R version 4.0.2"
Loading required package: sgeostat

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

sROC 0.1-2 loaded



In [2]:
conn <- DBI::dbConnect(odbc::odbc(), 
                       Driver = "SQL Server", 
                       Server = "DUSHI-DELL\\SQLEXPRESS",
                       Database = "Fraud Detection",
                       Trusted_Connection = "True")

In [3]:
df_fraud<-dbGetQuery(conn,'SELECT*FROM"Fraud Detection"."dbo"."train"')

In [5]:
head(df_fraud)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
,<int>,<int>,<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<dbl>,...,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2987000,0,86400,68.5,W,13926,NA,150,discover,142,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2987001,0,86401,29.0,W,2755,404,150,mastercard,102,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,2987002,0,86469,59.0,W,4663,490,150,visa,166,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,2987003,0,86499,50.0,W,18132,567,150,mastercard,117,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,2987004,0,86506,50.0,H,4497,514,150,mastercard,102,...,samsung browser 6.2,32,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
6,2987005,0,86510,49.0,W,5937,555,150,visa,226,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Descriptive Statistics

In [6]:
summary(df_fraud)

 TransactionID        isFraud        TransactionDT      TransactionAmt    
 Min.   :2987000   Min.   :0.00000   Min.   :   86400   Min.   :    0.25  
 1st Qu.:3134635   1st Qu.:0.00000   1st Qu.: 3027058   1st Qu.:   43.32  
 Median :3282270   Median :0.00000   Median : 7306528   Median :   68.77  
 Mean   :3282270   Mean   :0.03499   Mean   : 7372311   Mean   :  135.03  
 3rd Qu.:3429904   3rd Qu.:0.00000   3rd Qu.:11246620   3rd Qu.:  125.00  
 Max.   :3577539   Max.   :1.00000   Max.   :15811131   Max.   :31937.39  
                                                                          
  ProductCD             card1           card2           card3      
 Length:590540      Min.   : 1000   Min.   :100.0   Min.   :100.0  
 Class :character   1st Qu.: 6019   1st Qu.:214.0   1st Qu.:150.0  
 Mode  :character   Median : 9678   Median :361.0   Median :150.0  
                    Mean   : 9899   Mean   :362.6   Mean   :153.2  
                    3rd Qu.:14184   3rd Qu.:512.0   3rd Qu.:

In [7]:
### characters
strlst <- NULL
numlst <- NULL
for (v in names(df_fraud)) {
    if(typeof(df_fraud[[v]])=='character' | is.factor(df_fraud[[v]])) {
        strlst <- c(strlst,v) 
    } else {
        numlst <- c(numlst, v)
    }
}
strlst
numlst

[1] "ProductCD"     "card4"         "card6"         "P_emaildomain"
 [5] "R_emaildomain" "M1"            "M2"            "M3"           
 [9] "M4"            "M5"            "M6"            "M7"           
[13] "M8"            "M9"            "id_12"         "id_15"        
[17] "id_16"         "id_23"         "id_27"         "id_28"        
[21] "id_29"         "id_30"         "id_31"         "id_33"        
[25] "id_34"         "id_35"         "id_36"         "id_37"        
[29] "id_38"         "DeviceType"    "DeviceInfo"

[1] "TransactionID"  "isFraud"        "TransactionDT"  "TransactionAmt"
  [5] "card1"          "card2"          "card3"          "card5"         
  [9] "addr1"          "addr2"          "dist1"          "dist2"         
 [13] "C1"             "C2"             "C3"             "C4"            
 [17] "C5"             "C6"             "C7"             "C8"            
 [21] "C9"             "C10"            "C11"            "C12"           
 [25] "C13"            "C14"            "D1"             "D2"            
 [29] "D3"             "D4"             "D5"             "D6"            
 [33] "D7"             "D8"             "D9"             "D10"           
 [37] "D11"            "D12"            "D13"            "D14"           
 [41] "D15"            "V1"             "V2"             "V3"            
 [45] "V4"             "V5"             "V6"             "V7"            
 [49] "V8"             "V9"             "V10"            "V11"           
 [53] "V12"            "V13"            "V14"            "V15"           
 [57] "V16"            "V17"            "V18"            "V19"           
 [61] "V20"            "V21"            "V22"            "V23"           
 [65] "V24"            "V25"            "V26"            "V27"           
 [69] "V28"            "V29"            "V30"            "V31"           
 [73] "V32"            "V33"            "V34"            "V35"           
 [77] "V36"            "V37"            "V38"            "V39"           
 [81] "V40"            "V41"            "V42"            "V43"           
 [85] "V44"            "V45"            "V46"            "V47"           
 [89] "V48"            "V49"            "V50"            "V51"           
 [93] "V52"            "V53"            "V54"            "V55"           
 [97] "V56"            "V57"            "V58"            "V59"           
[101] "V60"            "V61"            "V62"            "V63"           
[105] "V64"            "V65"            "V66"            "V67"           
[109] "V68"            "V69"            "V70"            "V71"           
[113] "V72"            "V73"            "V74"            "V75"           
[117] "V76"            "V77"            "V78"            "V79"           
[121] "V80"            "V81"            "V82"            "V83"           
[125] "V84"            "V85"            "V86"            "V87"           
[129] "V88"            "V89"            "V90"            "V91"           
[133] "V92"            "V93"            "V94"            "V95"           
[137] "V96"            "V97"            "V98"            "V99"           
[141] "V100"           "V101"           "V102"           "V103"          
[145] "V104"           "V105"           "V106"           "V107"          
[149] "V108"           "V109"           "V110"           "V111"          
[153] "V112"           "V113"           "V114"           "V115"          
[157] "V116"           "V117"           "V118"           "V119"          
[161] "V120"           "V121"           "V122"           "V123"          
[165] "V124"           "V125"           "V126"           "V127"          
[169] "V128"           "V129"           "V130"           "V131"          
[173] "V132"           "V133"           "V134"           "V135"          
[177] "V136"           "V137"           "V138"           "V139"          
[181] "V140"           "V141"           "V142"           "V143"          
[185] "V144"           "V145"           "V146"           "V147"          
[189] "V148"           "V149"           "V150"           "V151"          
[193] "V152"           "V153"           "V154"           "V155"          
[197] "V156"           "V157"           "V158"           "V159"          
[201] "V160"           "V161"           "V162"           "V163"          
[205] "V164"           "V165"           "V166"           "V167"          
[209] "V168"           "V169"           "V170"           "V171"          
[213] "V172"           "V173"           "V174"           "V175"          
[217] 

In [8]:
df_fraud $addr1<- as.character (df_fraud $addr1)
df_fraud $addr2<- as.character (df_fraud $addr2)

In [9]:
summary(df_fraud)

 TransactionID        isFraud        TransactionDT      TransactionAmt    
 Min.   :2987000   Min.   :0.00000   Min.   :   86400   Min.   :    0.25  
 1st Qu.:3134635   1st Qu.:0.00000   1st Qu.: 3027058   1st Qu.:   43.32  
 Median :3282270   Median :0.00000   Median : 7306528   Median :   68.77  
 Mean   :3282270   Mean   :0.03499   Mean   : 7372311   Mean   :  135.03  
 3rd Qu.:3429904   3rd Qu.:0.00000   3rd Qu.:11246620   3rd Qu.:  125.00  
 Max.   :3577539   Max.   :1.00000   Max.   :15811131   Max.   :31937.39  
                                                                          
  ProductCD             card1           card2           card3      
 Length:590540      Min.   : 1000   Min.   :100.0   Min.   :100.0  
 Class :character   1st Qu.: 6019   1st Qu.:214.0   1st Qu.:150.0  
 Mode  :character   Median : 9678   Median :361.0   Median :150.0  
                    Mean   : 9899   Mean   :362.6   Mean   :153.2  
                    3rd Qu.:14184   3rd Qu.:512.0   3rd Qu.:

In [4]:
tab1 <- suppressWarnings(mechkar::Table1(data=df_fraud[,1:ncol(df_fraud)],y="isFraud"))

  |===================================================================   |  96%

ERROR: Error: cannot allocate vector of size 4.3 Mb


In [5]:
tab1

ERROR: Error in eval(expr, envir, enclos): object 'tab1' not found


In [6]:
suppressWarnings(mechkar::exploreData(data=df_fraud, y="isFraud",dir="C:/Users/Rita/DScourse/exp_Fraud"))

  |                                                                      |   0%

ERROR: Error in s[[lv]]: subscript out of bounds
